In [1]:
import requests 
import datetime
import pandas as pd
import altair as alt


from dotenv import load_dotenv
import os

load_dotenv()

# api_key=os.getenv("WEATHERAPI_APIKEY")

True

In [2]:
# Historical data does not work with the free key
# response = requests.get(f"https://api.weatherapi.com/v1/current.json?key={api_key}&q=Berlin&dt=2015-01-01")
# berlin = response.json()

In [3]:
# I thought about doing an API-Call, but downloading the csv was just so much easier. 

# response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.405&start_date=2015-06-16&end_date=2025-06-15&daily=temperature_2m_mean,rain_sum,snowfall_sum,sunshine_duration')

# berlin = response.json()

# response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=40.6782&longitude=73.9442&start_date=2015-06-16&end_date=2025-06-15&daily=temperature_2m_mean,rain_sum,snowfall_sum,sunshine_duration')

# new_york = response.json()

Data source: https://open-meteo.com/

This describtion works for both dataframes - one will have berlin added, the other one will have brooklyn added

day: date of the measures
temperature: mean temperature on that day in Celcius
rain: sum of rain on that day in millimeter
snowfall: sum of snowfall on that day in Centimeter
sunshine: duration of sunshine in that day in hours (converted von seconds to hours)

In [4]:
df = pd.read_csv('raw/weather_ berlin.csv', header=2)

In [5]:
# rename columns

(
    df
    .rename(
        columns={
            'time': 'day', 
            'temperature_2m_mean (°C)' : 'temperature', 
            'rain_sum (mm)' : 'rain', 
            'snowfall_sum (cm)': 'snowfall', 
            'sunshine_duration (s)':'sunshine'
        }, 
        inplace=True
    )
)

In [6]:
df

,day,temperature,rain,snowfall,sunshine
0,2015-06-16,12.3,0.0,0.0,17109.14
1,2015-06-17,15.2,0.5,0.0,46747.82
2,2015-06-18,15.9,4.2,0.0,30454.32
3,2015-06-19,14.3,0.1,0.0,52821.24
4,2015-06-20,13.8,3.1,0.0,46659.14
...,...,...,...,...,...
3648,2025-06-11,15.8,0.9,0.0,48035.18
3649,2025-06-12,16.8,0.0,0.0,55367.75
3650,2025-06-13,19.1,0.0,0.0,55407.55
3651,2025-06-14,23.4,0.0,0.0,55486.68


In [7]:
# calculate sunshine from seconds to hours

df['sunshine_hours'] = df['sunshine'] / 3600
df

,day,temperature,rain,snowfall,sunshine,sunshine_hours
0,2015-06-16,12.3,0.0,0.0,17109.14,4.752539
1,2015-06-17,15.2,0.5,0.0,46747.82,12.985506
2,2015-06-18,15.9,4.2,0.0,30454.32,8.459533
3,2015-06-19,14.3,0.1,0.0,52821.24,14.672567
4,2015-06-20,13.8,3.1,0.0,46659.14,12.960872
...,...,...,...,...,...,...
3648,2025-06-11,15.8,0.9,0.0,48035.18,13.343106
3649,2025-06-12,16.8,0.0,0.0,55367.75,15.379931
3650,2025-06-13,19.1,0.0,0.0,55407.55,15.390986
3651,2025-06-14,23.4,0.0,0.0,55486.68,15.412967


In [8]:
# add rolling averages to file for visualisations 

df['rain_7d_avg'] = df['rain'].rolling(window=7).mean()
df['sunshine_hours_7d_avg'] = df['sunshine'].rolling(window=7).mean()
df['snowfall_7d_avg'] = df['snowfall'].rolling(window=7).mean()
df['rain_30d_avg'] = df['rain'].rolling(window=30).mean()
df['sunshine_hours_30d_avg'] = df['sunshine'].rolling(window=30).mean()
df['snowfall_30d_avg'] = df['snowfall'].rolling(window=30).mean()

In [9]:
df.describe()

,temperature,rain,snowfall,sunshine,sunshine_hours,rain_7d_avg,sunshine_hours_7d_avg,snowfall_7d_avg,rain_30d_avg,sunshine_hours_30d_avg,snowfall_30d_avg
count,3653.00000,3653.000000,3653.000000,3653.000000,3653.000000,3647.000000,3647.000000,3647.000000,3624.000000,3624.000000,3624.000000
mean,10.97101,1.559212,0.065918,26891.106564,7.469752,1.560069,26865.223823,0.066027,1.554312,26779.151677,0.066446
std,7.67585,3.679876,0.394038,17248.800728,4.791334,1.741802,13731.429388,0.213182,0.965854,12685.437065,0.142440
min,-10.30000,0.000000,0.000000,0.000000,0.000000,0.000000,54.011429,0.000000,0.043333,4460.590667,0.000000
25%,5.00000,0.000000,0.000000,12052.110000,3.347808,0.335714,14244.920000,0.000000,0.913333,14451.904500,0.000000
50%,10.60000,0.200000,0.000000,28027.920000,7.785533,1.100000,26913.732857,0.000000,1.396667,27600.725667,0.000000
75%,17.30000,1.600000,0.000000,42145.390000,11.707053,2.242857,39133.631429,0.000000,1.983333,38611.521417,0.051333
max,30.10000,109.600000,10.780000,55586.050000,15.440569,21.742857,54612.767143,2.760000,7.786667,49450.967333,0.924000


In [10]:
# save cleaned file with calculations

df.to_csv('processed/berlin_cleaned.csv', index=False)

In [11]:
# do the same for the Brooklyn weather to have the same data

df2 = pd.read_csv('raw/weather_brooklyn.csv', header=2)

In [12]:
df2

,time,rain_sum (mm),snowfall_sum (cm),sunshine_duration (s),temperature_2m_mean (°C)
0,2015-06-16,8.9,0.0,18014.44,21.2
1,2015-06-17,0.0,0.0,34487.41,21.6
2,2015-06-18,0.0,0.0,14856.77,18.7
3,2015-06-19,0.3,0.0,38022.73,22.0
4,2015-06-20,0.1,0.0,7130.58,20.5
...,...,...,...,...,...
3648,2025-06-11,0.1,0.0,48881.35,23.2
3649,2025-06-12,0.0,0.0,50199.27,26.7
3650,2025-06-13,0.1,0.0,43295.38,25.0
3651,2025-06-14,4.9,0.0,2993.10,18.8


In [13]:
# rename columns 

(
    df2
    .rename(
        columns={
            'time': 'day', 
            'temperature_2m_mean (°C)' : 'temperature', 
            'rain_sum (mm)' : 'rain', 
            'snowfall_sum (cm)': 'snowfall', 
            'sunshine_duration (s)':'sunshine'
        }, 
        inplace=True
    )
)
df2

,day,rain,snowfall,sunshine,temperature
0,2015-06-16,8.9,0.0,18014.44,21.2
1,2015-06-17,0.0,0.0,34487.41,21.6
2,2015-06-18,0.0,0.0,14856.77,18.7
3,2015-06-19,0.3,0.0,38022.73,22.0
4,2015-06-20,0.1,0.0,7130.58,20.5
...,...,...,...,...,...
3648,2025-06-11,0.1,0.0,48881.35,23.2
3649,2025-06-12,0.0,0.0,50199.27,26.7
3650,2025-06-13,0.1,0.0,43295.38,25.0
3651,2025-06-14,4.9,0.0,2993.10,18.8


In [14]:
# calculate sunshine from seconds to hours
df2['sunshine_hours'] = df2['sunshine'] / 3600
df2

,day,rain,snowfall,sunshine,temperature,sunshine_hours
0,2015-06-16,8.9,0.0,18014.44,21.2,5.004011
1,2015-06-17,0.0,0.0,34487.41,21.6,9.579836
2,2015-06-18,0.0,0.0,14856.77,18.7,4.126881
3,2015-06-19,0.3,0.0,38022.73,22.0,10.561869
4,2015-06-20,0.1,0.0,7130.58,20.5,1.980717
...,...,...,...,...,...,...
3648,2025-06-11,0.1,0.0,48881.35,23.2,13.578153
3649,2025-06-12,0.0,0.0,50199.27,26.7,13.944242
3650,2025-06-13,0.1,0.0,43295.38,25.0,12.026494
3651,2025-06-14,4.9,0.0,2993.10,18.8,0.831417


In [15]:
df2['rain_7d_avg'] = df2['rain'].rolling(window=7).mean()
df2['sunshine_hours_7d_avg'] = df2['sunshine'].rolling(window=7).mean()
df2['snowfall_7d_avg'] = df2['snowfall'].rolling(window=7).mean()
df2['rain_30d_avg'] = df2['rain'].rolling(window=30).mean()
df2['sunshine_hours_30d_avg'] = df2['sunshine'].rolling(window=30).mean()
df2['snowfall_30d_avg'] = df2['snowfall'].rolling(window=30).mean()

In [16]:
df2.describe()

,rain,snowfall,sunshine,temperature,sunshine_hours,rain_7d_avg,sunshine_hours_7d_avg,snowfall_7d_avg,rain_30d_avg,sunshine_hours_30d_avg,snowfall_30d_avg
count,3653.000000,3653.000000,3653.000000,3653.000000,3653.000000,3647.000000,3647.000000,3647.000000,3624.000000,3624.000000,3624.000000
mean,3.275198,0.146573,29823.879636,12.659869,8.284411,3.274370,29834.209857,0.146814,3.257888,29803.278338,0.147746
std,7.655594,1.267931,14828.393538,9.147561,4.118998,3.111781,9285.319056,0.548066,1.599828,7637.893975,0.330264
min,0.000000,0.000000,0.000000,-14.900000,0.000000,0.000000,1503.737143,0.000000,0.066667,14978.585000,0.000000
25%,0.000000,0.000000,21357.990000,5.200000,5.932775,0.942857,22710.205714,0.000000,2.113333,23576.834833,0.000000
50%,0.000000,0.000000,32661.840000,12.600000,9.072733,2.514286,29352.191429,0.000000,3.046667,28936.781833,0.000000
75%,2.800000,0.000000,41535.690000,21.000000,11.537692,4.657143,37002.982857,0.000000,4.128333,36130.109500,0.079333
max,110.500000,28.000000,50400.000000,30.400000,14.000000,28.057143,50350.338571,5.210000,9.630000,46868.613000,2.069667


In [17]:
# save cleaned file with calculations

df.to_csv('processed/brooklyn_cleaned.csv', index=False)